### developing chatbot plan : 
first i will extract the data from .pdf by mannual...
then i will tokenize the each words.

the model will be simple llm model . i will use "Attention is what you need" paper's diagram to create the model
from scratch.. 

for the model developement : 

first i will create the :  
1) multihead attention (Query,Key,Value) function 
2) text embedding function
3) positional encoding function

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import re
import string
import json
import os
from module.createData import CreateDataset
from module.readData import ReadDataset
from module.cuda import getDevice
from module.bot import Chatbot
from module.Transformer import Transformer
from module.TrainModel import trainModel
from module.TestModel import Test

from module.VD import VectorDatabase

In [23]:
with open('asset/data/data.txt', 'r',encoding='utf-8') as f:
    data = f.read()

In [24]:
data

"1.2 \nFINANCIAL POLICY OBJECTIVES AND STRATEGIES \n STATEMENT \nThe presentation and preparation of the Territory’s Budget is provided for in sections 11 and \n11A of the Financial Management Act 1996 (the Act).   \nThe purpose of the financial policy objectives and strategies statement is to make transparent \nthe Government’s financial strategies and to establish a benchmark for evaluating the \nGovernment’s conduct of financial policy.  The statement is also consistent with section \n11(1)(a) of the Act. \nStrategic Priorities and Financial Policy \nIn this budget, the Government continues its commitment to the principles of responsible \nfinancial management. \nThe financial objectives and the key measures below outline the Government’s financial \npolicy.  Strategic priorities, as they relate to the Territory’s Budget, are summarised as: \n• maintain a balanced budget over the economic cycle; \n• maintain low levels of debt; \n• provide the highest possible standard of government

In [25]:
getDevice = getDevice()

In [26]:
def ExtractKeyInfo(InputFilePath, OutputFilePath):
    if not os.path.exists(InputFilePath):
        raise FileNotFoundError(f"Input file {InputFilePath} not found.")
    
    with open(InputFilePath, 'r', encoding='utf-8') as F:
        Document = F.read()
    
    Pages = re.split(r'2005-06 Budget Paper No. 3\s*(\d+)\s*Financial Policy Objectives and Strategies Statement', Document)
    PageContents = []
    CurrentPage = 5
    for I in range(0, len(Pages), 2):
        Content = Pages[I].strip()
        if Content:
            PageContents.append({'Page': CurrentPage, 'Content': Content})
        if I+1 < len(Pages):
            CurrentPage = int(Pages[I+1])
    
    KeyInfo = []
    Strategic = re.search(r'Strategic priorities, as they relate to the Territory’s Budget, are summarised as:(.*?)\. The 2005-06 Budget and Forward Estimates', Document, re.S)
    if Strategic:
        Priorities = re.findall(r'• (.*?);', Strategic.group(1))
        for P in Priorities:
            KeyInfo.append({'Info': P.strip(), 'Source': f'Page {CurrentPage}'})
    
    KeyInfo.extend([
        {'Info': 'The Budget provides an aggregate surplus over four years. Cash reserves reducing through to 2006-07 but forecast to grow in 2008-09, general government borrowings not increased.', 'Source': 'Page 5'},
        {'Info': 'Resources directed into health, education, child and family support, meeting demand of growing and ageing population, targeted at vulnerable.', 'Source': 'Page 5'},
        {'Info': 'Investments in Shaping Our Territory plan, ACT Energy Wise Program, Asbestos Assessment Task Force.', 'Source': 'Page 5'},
        {'Info': 'Financial Objectives and Measures (Table 1.2.1): Short Term: Maintain Triple A credit rating, low debt, infrastructure, long-term liabilities, strategic capital works, minimise risk, high standard services. Long Term: Balanced budget, 90% superannuation coverage by 2039-2040, reduce GGS debt, net interest cost <0, mitigate risk, taxation % GSP, increase net assets.', 'Source': 'Page 6'},
        {'Info': 'GGS Operating Result (Table 1.2.2): 2002-03 154.6m, 2003-04 70.5m, 2004-05 -52.2m, 2005-06 91.5m, 2006-07 0.9m, 2007-08 39.3m, 2008-09 73.3m. Aggregate 2002-03 to 2005-06 185.7m, 2005-06 to 2008-09 22.0m.', 'Source': 'Page 6'},
        {'Info': 'Capital Infrastructure (Table 1.2.3): Non-Current WIP 136m to 41m, Property Plant Equipment 9,905m to 11,034m, Sub Total 10,040m to 11,075m.', 'Source': 'Page 6'},
        {'Info': 'Net Interest Cost % Own-Source Revenue (Table 1.2.4): -1.9% to -0.1%.', 'Source': 'Page 7'},
        {'Info': 'Taxation % GSP (Table 1.2.5): 4.1% to 4.4%.', 'Source': 'Page 8'},
        {'Info': 'Total Territory Net Assets (Table 1.2.6): 9,304m to 9,878m.', 'Source': 'Page 8'},
        {'Info': 'Superannuation Liabilities Funding (Table 1.2.7): 58% in 2005 to 67% in 2009.', 'Source': 'Page 9'},
        {'Info': 'Principles of Responsible Financial Management: Prudent liabilities, balanced expenses, net worth buffer, fiscal risks, stable taxing, full disclosure.', 'Source': 'Page 9'},
        {'Info': 'Infrastructure investments: Stromlo Forest Park, Primary and Pre-School in East Gungahlin, Quamby replacement, Gungahlin Drive Extension, Alexander Maconachie Centre.', 'Source': 'Page 10'},
    ])
    
    with open(OutputFilePath, 'w', encoding='utf-8') as F:
        json.dump(KeyInfo, F, indent=4)
    
    return KeyInfo


In [27]:
def GenerateDatasetFile(KeyInfo, DatasetFilePath):
    ansPair = [
        ("What is the budget strategy?", "The Budget provides an aggregate surplus over four years. Cash reserves reducing through to 2006-07 but forecast to grow in 2008-09, general government borrowings not increased. (Source: Page 5)"),
        ("What about debt levels?", "Financial Objectives and Measures (Table 1.2.1): Short Term: Maintain Triple A credit rating, low debt, infrastructure, long-term liabilities, strategic capital works, minimise risk, high standard services. Long Term: Balanced budget, 90% superannuation coverage by 2039-2040, reduce GGS debt, net interest cost <0, mitigate risk, taxation % GSP, increase net assets. (Source: Page 6)"),
        ("What are the infrastructure investments?", "Infrastructure investments: Stromlo Forest Park, Primary and Pre-School in East Gungahlin, Quamby replacement, Gungahlin Drive Extension, Alexander Maconachie Centre. (Source: Page 10)"),
        ("What is the net interest cost?", "Net Interest Cost % Own-Source Revenue (Table 1.2.4): -1.9% to -0.1%. (Source: Page 7)"),
        ("How is taxation managed?", "Taxation % GSP (Table 1.2.5): 4.1% to 4.4%. (Source: Page 8)"),
        ("What about superannuation liabilities?", "Superannuation Liabilities Funding (Table 1.2.7): 58% in 2005 to 67% in 2009. (Source: Page 9)"),
        ("What are the principles of financial management?", "Principles of Responsible Financial Management: Prudent liabilities, balanced expenses, net worth buffer, fiscal risks, stable taxing, full disclosure. (Source: Page 9)"),
        ("What are the strategic priorities?", "maintain a balanced budget over the economic cycle (Source: Page 5)"),
        ("What are the services prioritized?", "Resources directed into health, education, child and family support, meeting demand of growing and ageing population, targeted at vulnerable. (Source: Page 5)"),
        ("What are the investment programs?", "Investments in Shaping Our Territory plan, ACT Energy Wise Program, Asbestos Assessment Task Force. (Source: Page 5)")
    ]
    
    with open(DatasetFilePath, 'w', encoding='utf-8') as F:
        for Question, Answer in ansPair:
            F.write(f'"{Question}", "{Answer}"\n')
    
    return ansPair

### Datapath initialize

In [28]:
InputFilePath = 'asset/data/data.txt'
JsonFilePath = 'asset/preprocessed/extractedData.json'
DatasetFilePath = 'asset/preprocessed/dataset.txt'

In [29]:
if not os.path.exists(JsonFilePath):
    KeyInfo = ExtractKeyInfo(InputFilePath, JsonFilePath)
else:
    with open(JsonFilePath, 'r', encoding='utf-8') as F:
        KeyInfo = json.load(F)


if not os.path.exists(DatasetFilePath):
    GenerateDatasetFile(KeyInfo, DatasetFilePath)

### Initializing vector DB

In [30]:
Db = VectorDatabase(KeyInfo)

### vocabulary creation

In [31]:
vocab = set()

for Item in KeyInfo:
    vocab.update(Item['Info'].lower().translate(str.maketrans('', '', string.punctuation)).split())
vocab.update(['<START>', '<END>', '<PAD>'])

lti = {Word: Idx for Idx, Word in enumerate(vocab)}

itl = {Idx: Word for Word, Idx in lti.items()}

In [32]:
print(lti)

{'coverage': 0, 'prudent': 1, 'vulnerable': 2, 'population': 3, 'taxation': 4, 'in': 5, 'into': 6, 'and': 7, 'table': 8, '200304': 9, 'gungahlin': 10, '200405': 11, '9304m': 12, '67': 13, 'through': 14, 'financial': 15, 'a': 16, 'aggregate': 17, 'infrastructure': 18, '522m': 19, 'drive': 20, '733m': 21, '41m': 22, 'years': 23, 'provides': 24, 'force': 25, 'capital': 26, 'grow': 27, 'plan': 28, 'assets': 29, 'operating': 30, 'an': 31, 'family': 32, 'longterm': 33, 'objectives': 34, '11034m': 35, 'primary': 36, 'maconachie': 37, 'result': 38, '200708': 39, 'risks': 40, 'debt': 41, 'assessment': 42, '200809': 43, 'the': 44, 'act': 45, 'resources': 46, '393m': 47, 'worth': 48, 'directed': 49, 'our': 50, 'reserves': 51, 'works': 52, 'preschool': 53, 'of': 54, 'budget': 55, 'cash': 56, 'term': 57, 'funding': 58, 'increase': 59, 'ggs': 60, 'alexander': 61, 'taxing': 62, '<PAD>': 63, 'park': 64, 'strategic': 65, 'but': 66, 'disclosure': 67, '200203': 68, 'rating': 69, 'equipment': 70, 'revenue

In [33]:
print(itl)

{0: 'coverage', 1: 'prudent', 2: 'vulnerable', 3: 'population', 4: 'taxation', 5: 'in', 6: 'into', 7: 'and', 8: 'table', 9: '200304', 10: 'gungahlin', 11: '200405', 12: '9304m', 13: '67', 14: 'through', 15: 'financial', 16: 'a', 17: 'aggregate', 18: 'infrastructure', 19: '522m', 20: 'drive', 21: '733m', 22: '41m', 23: 'years', 24: 'provides', 25: 'force', 26: 'capital', 27: 'grow', 28: 'plan', 29: 'assets', 30: 'operating', 31: 'an', 32: 'family', 33: 'longterm', 34: 'objectives', 35: '11034m', 36: 'primary', 37: 'maconachie', 38: 'result', 39: '200708', 40: 'risks', 41: 'debt', 42: 'assessment', 43: '200809', 44: 'the', 45: 'act', 46: 'resources', 47: '393m', 48: 'worth', 49: 'directed', 50: 'our', 51: 'reserves', 52: 'works', 53: 'preschool', 54: 'of', 55: 'budget', 56: 'cash', 57: 'term', 58: 'funding', 59: 'increase', 60: 'ggs', 61: 'alexander', 62: 'taxing', 63: '<PAD>', 64: 'park', 65: 'strategic', 66: 'but', 67: 'disclosure', 68: '200203', 69: 'rating', 70: 'equipment', 71: 'rev

### Transformer parameters

In [34]:
DModel = 512
FfnHidden = 2048
NumHeads = 8
DropProb = 0.1
NumLayers = 6
MaxSequenceLength = 200
VocabSize = len(vocab)
StartToken ='<START>'
EndToken ='<END>'
PaddingToken ='<PAD>'

In [35]:
model = Transformer(DModel, FfnHidden, NumHeads, DropProb, NumLayers, MaxSequenceLength, VocabSize, lti, StartToken, EndToken, PaddingToken)
model.to(getDevice)

Transformer(
  (Encoder): Encoder(
    (SentenceEmbedding): SentenceEmbedding(
      (Embedding): Embedding(177, 512)
      (PositionEncoder): PE()
      (Dropout): Dropout(p=0.1, inplace=False)
    )
    (Layers): SequentialEncoder(
      (0): EncoderLayer(
        (Attention): MultiHeadAttention(
          (QkvLayer): Linear(in_features=512, out_features=1536, bias=True)
          (LinearLayer): Linear(in_features=512, out_features=512, bias=True)
        )
        (Norm1): LayerNormalization()
        (Dropout1): Dropout(p=0.1, inplace=False)
        (Ffn): PositionwiseFeedForward(
          (Linear1): Linear(in_features=512, out_features=2048, bias=True)
          (Linear2): Linear(in_features=2048, out_features=512, bias=True)
          (Relu): ReLU()
          (Dropout): Dropout(p=0.1, inplace=False)
        )
        (Norm2): LayerNormalization()
        (Dropout2): Dropout(p=0.1, inplace=False)
      )
      (1): EncoderLayer(
        (Attention): MultiHeadAttention(
          

In [36]:
Dataset = CreateDataset(DatasetFilePath,lti,MaxSequenceLength)

In [37]:
print(Dataset)

[(tensor([152,  63,  63,  44,  55,  63,  77,  63,  63,  63,  63,  63,  63,  63,
         63,  63,  63,  63,  63,  63,  63,  63,  63,  63,  63,  63,  63,  63,
         63,  63,  63,  63,  63,  63,  63,  63,  63,  63,  63,  63,  63,  63,
         63,  63,  63,  63,  63,  63,  63,  63,  63,  63,  63,  63,  63,  63,
         63,  63,  63,  63,  63,  63,  63,  63,  63,  63,  63,  63,  63,  63,
         63,  63,  63,  63,  63,  63,  63,  63,  63,  63,  63,  63,  63,  63,
         63,  63,  63,  63,  63,  63,  63,  63,  63,  63,  63,  63,  63,  63,
         63,  63,  63,  63,  63,  63,  63,  63,  63,  63,  63,  63,  63,  63,
         63,  63,  63,  63,  63,  63,  63,  63,  63,  63,  63,  63,  63,  63,
         63,  63,  63,  63,  63,  63,  63,  63,  63,  63,  63,  63,  63,  63,
         63,  63,  63,  63,  63,  63,  63,  63,  63,  63,  63,  63,  63,  63,
         63,  63,  63,  63,  63,  63,  63,  63,  63,  63,  63,  63,  63,  63,
         63,  63,  63,  63,  63,  63,  63,  63,  63,  63,  63,

In [ ]:
trainModel(model, Dataset,lti,Epochs=10,BatchSize=2,LearningRate=0.0001)

TypeError: to() received an invalid combination of arguments - got (function), but expected one of:
 * (torch.device device = None, torch.dtype dtype = None, bool non_blocking = False, bool copy = False, *, torch.memory_format memory_format = None)
 * (torch.dtype dtype, bool non_blocking = False, bool copy = False, *, torch.memory_format memory_format = None)
 * (Tensor tensor, bool non_blocking = False, bool copy = False, *, torch.memory_format memory_format = None)


In [ ]:
TestQueries = ["What about debt?","Tell me about infrastructure.","What is the budget strategy?"]
TestResults = Test(model,lti ,itl, MaxSequenceLength, TestQueries)

In [ ]:
for Query, Response in TestResults:
    print(f"Query: {Query}\nResponse: {Response}\n")

## check q/a

In [ ]:
xc = Chatbot(Db,model,lti,itl,MaxSequenceLength,StartToken,EndToken,PaddingToken)

In [ ]:
print(xc.Ask("What about debt?"))


In [ ]:
print(xc.Ask("Tell me about infrastructure."))